In [1]:
import warnings
import requests # API 연결 라이브러리
from lxml import html # XML 파싱을 위한 라이브러리
from bs4 import BeautifulSoup, XMLParsedAsHTMLWarning
import requests, bs4
import pandas as pd
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

In [2]:
def saveSidoCode(serviceKey):
    """
    구조동물 조회를 위한 시도코드 정보를 가져오기 위한 함수입니다.

    Parameter
    -------
    serviceKey: 공공데이터포털에서 발급받은 인증키(Encoding)

    Return
    -------
    orgCdList: 시도코드가 담긴 리스트
    orgDownNmList: 시도명이 담긴 리스트
    """
    try:
        orgCdList = []
        orgDownNmList = []
        sidoEndPoint = "http://apis.data.go.kr/1543061/abandonmentPublicService_v2/sido_v2"
        pageRows = "20"
        pageNo = "1"
        dataType = "xml"
        sidoUrl = f"{sidoEndPoint}?serviceKey={serviceKey}&numOfRows={pageRows}&pageNo={pageNo}&_type={dataType}"
        resp = requests.get(sidoUrl)
        htmlObj = resp.text
        bsObj = bs4.BeautifulSoup(htmlObj, features="xml")
        orgCdItems = bsObj.find_all(name="orgCd")
        orgDownItems = bsObj.find_all(name="orgdownNm")
        for i in range(len(orgCdItems)):
            orgCdList.append(orgCdItems[i].text)
            orgDownNmList.append(orgDownItems[i].text)
        return orgCdList, orgDownNmList
    except Exception as e:
        print(str(e))

In [3]:
def loadRescueAnimal(serviceKey, orgCdList, orgDownNmList):
    """
    각 시도에 위치한 보호소에 보호중인(또는 보호중이었던) 구조동물들의 정보를
    조회하는 함수입니다.

    Parameter
    -------
    serviceKey: 공공데이터포털에서 발급받은 인증키(Encoding)
    orgCdList: 시도코드가 담긴 리스트
    orgDownNmList: 시도명이 담긴 리스트

    Return
    -------
    animalList: 구조동물들의 정보가 담긴 리스트
    """
    try:
        resEndPoint = "http://apis.data.go.kr/1543061/abandonmentPublicService_v2/abandonmentPublic_v2"
        dataType = "xml"
        numOfRows = 10
        animalList = []
        for i in range(len(orgCdList)):
            addr = orgCdList[i]
            addrNm = orgDownNmList[i]
            pageNo = 1
            resUrl = f"{resEndPoint}?serviceKey={serviceKey}&upr_cd={addr}&pageNo={pageNo}&numOfRows={numOfRows}&_type={dataType}"
            resp = requests.get(resUrl)
            htmlObj = resp.text
            bsObj = bs4.BeautifulSoup(htmlObj, features="xml")

            # 전체 페이지 구하기
            totalCnt = bsObj.find(name="totalCount").text
            totalPage = int(int(totalCnt) / numOfRows) + 1

            # 페이지를 반복하며 데이터 수집 후 리스트에 저장
            for page in range(totalPage):
                resUrl = f"{resEndPoint}?serviceKey={serviceKey}&upr_cd={addr}&pageNo={page}&numOfRows={numOfRows}&_type={dataType}"
                animalItems = bsObj.find_all(name="item")
                for j in range(len(animalItems)):
                    happenDt = animalItems[j].find(name='happenDt').text
                    happenPlace = animalItems[j].find(name='happenPlace').text
                    animalGbn = animalItems[j].find(name='upKindCd').text
                    if animalGbn == '417000':
                        animalKind = "개"
                    elif animalGbn == '422400':
                        animalKind = "고양이"
                    else:
                        animalKind = "기타"
                    kindNm = animalItems[j].find(name='kindNm').text
                    colorCd = animalItems[j].find(name='colorCd').text
                    animalAge = animalItems[j].find(name='age').text
                    animalWeight = animalItems[j].find(name='weight').text
                    noticeNo = animalItems[j].find(name='noticeNo').text
                    noticeSdt = animalItems[j].find(name='noticeSdt').text
                    noticeEdt = animalItems[j].find(name='noticeEdt').text
                    processState = animalItems[j].find(name='processState').text
                    sexCd = animalItems[j].find(name='sexCd').text
                    if sexCd == 'M':
                        gender = "수컷"
                    elif sexCd == 'F':
                        gender = "암컷"
                    else:
                        gender = "미상"
                    neuterYn = animalItems[j].find(name='neuterYn').text
                    if neuterYn == 'U':
                        neuterYn = "미상"
                    specialMark = animalItems[j].find(name='specialMark').text
                    careNm = animalItems[j].find(name='careNm').text
                    careTel = animalItems[j].find(name='careTel').text
                    careAddr = animalItems[j].find(name='careAddr').text
                    careOwnerNm = animalItems[j].find(name='careOwnerNm').text
                    orgNm = animalItems[j].find(name='orgNm').text
                    updTm = animalItems[0].find(name='updTm').text[:-2]
                    animalList.append([addrNm, happenDt, happenPlace, animalKind, kindNm,
                                    colorCd, animalAge, animalWeight, noticeNo,
                                    noticeSdt, noticeEdt, processState, gender,
                                    neuterYn, specialMark, careNm, careTel, careAddr,
                                    careOwnerNm, orgNm, updTm])
        return animalList
    except Exception as e:
        print(str(e))

In [ ]:
# serviceKey = "serviceKey" # 공공데이터 인코딩 인증키
orgCdList, orgDownNmList = saveSidoCode(serviceKey)
animalData = loadRescueAnimal(serviceKey, orgCdList, orgDownNmList)

In [5]:
columnList = ["시도", "접수일", "발견장소", "축종코드", "품종", "색상", "나이",
              "체중", "공고번호", "공고시작일", "공고종료일", "상태", "성별",
              "중성여부", "특징", "보호소이름", "보호소전화번호", "보호장소",
              "보호소대표자", "관할기관", "수정일"]
animalDf = pd.DataFrame(animalData, columns=columnList)
animalDf

,시도,접수일,발견장소,축종코드,품종,색상,나이,체중,공고번호,공고시작일,...,상태,성별,중성여부,특징,보호소이름,보호소전화번호,보호장소,보호소대표자,관할기관,수정일
0,서울특별시,20250407,서울 구로구 고척로49길 20 (고척동) 인근,개,포메라니안,기타(흰/연갈),2020(년생),3.3(Kg),서울-구로-2025-00032,20250407,...,보호중,암컷,미상,양 슬개골. 숨 거칠게 쉼. 코 분홍. 얼굴 주변/귀 끝 연갈. 부정교합. 온순. ...,한국동물구조관리협회,031-867-9119,경기도 양주시 남면 감악산로 63-37,김철훈,서울특별시 구로구,2025-04-07 16:01:53
1,서울특별시,20250407,연남동 경의선 산책길,개,요크셔 테리어,갈색&검정,2010(년생),4.9(Kg),서울-마포-2025-00027,20250407,...,보호중,수컷,Y,나이가 있어보임 활력좋음,홍익동물병원,02-325-2026,서울특별시 마포구 독막로 45 (합정동),이대영,서울특별시 마포구,2025-04-07 16:01:53
2,서울특별시,20250406,서울 금천구 시흥대로 459 (독산동),개,믹스견,갈색&흰색,2018(년생),6.6(Kg),서울-금천-2025-00048,20250407,...,보호중,수컷,N,눈물 자국. 코 검정. 온순. 얌전. 꼬리 단미 안됨. 털 약간 때탐. 칩 있음.,한국동물구조관리협회,031-867-9119,경기도 양주시 남면 감악산로 63-37 (남면),김철훈,서울특별시 금천구,2025-04-07 16:01:53
3,서울특별시,20250406,"서울 구로구 개봉로3길 87 (개봉동, 개봉한진아파트)",고양이,한국 고양이,기타(갈/검/흰),2021(년생),3.4(Kg),서울-구로-2025-00031,20250407,...,보호중,암컷,미상,중성화 이표. 경계. 예민. 양 후지 마비. 우측 가슴에서 옆구리 쓸린 상처 및 탈...,한국동물구조관리협회,031-867-9119,경기도 양주시 남면 감악산로 63-37,김철훈,서울특별시 구로구,2025-04-07 16:01:53
4,서울특별시,20250406,현대교통종점 사거리(홍연2교 사거리),개,스탠다드 닥스훈트,검정&황갈색,2022(년생),10(Kg),서울-서대문-2025-00031,20250407,...,보호중,수컷,N,"사람 친화적, 콧등 위 상처, 각질이 많고 발톱이 길음",내품애센터,02-330-3821,서울특별시 서대문구 모래내로 333 (홍은동),윤희본,서울특별시 서대문구,2025-04-07 16:01:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7665,제주특별자치도,20250404,제주시 조천읍 북촌리539-1,개,믹스견,갈색,2021(년생),19.4(Kg),제주-제주-2025-00734,20250405,...,보호중,수컷,N,개체관리번호 0686 - 파란징목줄,제주 동물보호센터,064-710-4065,제주특별자치도 제주시 첨단동길 184-14 (용강동),김은주,제주특별자치도,2025-04-05 10:50:38
7666,제주특별자치도,20250404,서귀포시 남원읍 서성로636,개,믹스견,갈색,2025(년생),4.4(Kg),제주-제주-2025-00733,20250405,...,보호중,수컷,N,개체관리번호 0685 - 3개월,제주 동물보호센터,064-710-4065,제주특별자치도 제주시 첨단동길 184-14 (용강동),김은주,제주특별자치도,2025-04-05 10:50:38
7667,제주특별자치도,20250404,서귀포시 성산읍 신풍리373,개,믹스견,갈색,2024(년생),17.1(Kg),제주-제주-2025-00732,20250405,...,보호중,수컷,N,개체관리번호 0684,제주 동물보호센터,064-710-4065,제주특별자치도 제주시 첨단동길 184-14 (용강동),김은주,제주특별자치도,2025-04-05 10:50:38
7668,제주특별자치도,20250404,서귀포시 남원읍 신흥고수로375-7,개,믹스견,흰색,2025(60일미만)(년생),1.2(Kg),제주-제주-2025-00731,20250405,...,보호중,암컷,N,개체관리번호 0683 - 1개월,제주 동물보호센터,064-710-4065,제주특별자치도 제주시 첨단동길 184-14 (용강동),김은주,제주특별자치도,2025-04-05 10:50:38
